# Fixing a Pin Cell problem

## Scenario
* Given a model `models/pin_cell.imcnp` from dubious origins
    * Meant to represent a W17$\times$17 pin cell
* However, has a few Problems that need to be fixed first


## First, let's just print out the file.

In [6]:
!cat models/pin_cell.imcnp

Made up PWR pin Cell Problem
C dimensions, densities, and material compositions are based on the BEAVRS benchmark
C They have been rounded, and further simplified
C
C N. Horelik, B. Herman, B. Forget, and K. Smith. Benchmark for Evaluation and Validation of
C    Reactor Simulations (BEAVRS), v1.0.1. Proc. Int. Conf. Mathematics and Computational Methods
C    Applied to Nuc. Sci. & Eng., 2013. Sun Valley, Idaho
c
c uranium rod
1    1  -10.36   -1     101 -102 imp:n=1 
c gas gap
2    0            1  -2 101 -102 imp:n=1 
c cladding
3    10 -6.55     2  -3 101 -102 imp:n=1 
c water cell
5    2  -1.000    3     101 -102
                        103 -104
                        105 -106 imp:n=1 

1 CZ 0.39
2 CZ 0.40
3 CZ 0.46
101 PZ -0.63
102 PZ  0.63
103 PX -0.63
104 PX  0.63
105 PY -0.63
106 PY  0.63

c UO2 fuel 3.4% enriched
M1    92235.82c 0.011332
      92238.82c 0.321678
       8016.82c 0.66667
mt1   o2/u.20t  u/o2.30t
C water
M2     1001.81c 0.66
       8016.81c 0.33
mt2    lwtr.20t
C 

/home/mgale/miniforge3/envs/montepy_demo/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4281) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


# Plot

---
**NOTE**

You will need MCNP to plot geometries. MCNP is distributed through The [Radiation Safety Information Computational Center (RSICC)](https://rsicc.ornl.gov/) at Oak Ridge National Laboratory and cannot be provided in this workshop. The depicted plots are for information only.

---

![pin cell plot](figs/pin_cell.png)

# Improvement: Goals
* Add helium to gas gap
* Fix Zircaloy
   * Elemental cross sections are generally the wrong tool
   * Mixing different nuclear data libraries is a bad idea
* Fix (lack of) reflective boundaries
* Set this up for running Kcode

# Step 1: Import Python modules

In [7]:
# actually needed
import montepy
import numpy as np

# for pretty Jupyter web pages Usually not needed
from IPython.display import IFrame

# Step 2: Open the File in MontePy
1. Read the docs

In [8]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.input_parser.input_reader.html#montepy.input_parser.input_reader.read_input",
    1200,
    500,
)

2. This function does appear to be the correct one.
     * destination is required
     * `mcnp_version` is optional. We won't be using specific features that require changing this
     * replace is optional. Here the default seems like a safe bet.
### tip: this function is available at the top level: `montepy.read_input`

In [9]:
problem = montepy.read_input("models/pin_cell.imcnp")

# Step 3: Explore the Problem

## Goals
* print the following
   * the cells, surfaces, and materials in the problem
   * the version of MCNP this is designed for

In [10]:
IFrame("https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html", 800, 600)

# Pause for students to complete

In [11]:
print(problem.cells)
print(problem.surfaces)
print(problem.materials)
print("version", problem.mcnp_version)

Cells: [1, 2, 3, 5]
Surfaces: [1, 2, 3, 101, 102, 103, 104, 105, 106]
Materials: [1, 2, 10]
version (6, 3, 0)


# Goal 1: Fix Boundary Conditions
## Reminder
* Reflective boundaries are a property of the surface; not the cell
## Things to figure out:
1. How to get the specific surfaces:
   * Did you notice that planes (e.g., `PZ`) were only used as boundaries?
2. How to set a reflective boundary

# Task 1.1 Is there an easy way to get a surface by type?

In [12]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.surface_collection.html", 800, 500
)

## Let's explore these surface type generators.
* Hint: generators are a specific type of iterator
### Task: Print all surfaces of type `PZ`

# Pause for students to complete

In [13]:
# Task: Print every "PZ" type of surface
z_surfaces = problem.surfaces.pz  # Change this line
for surface in z_surfaces:
    print(repr(surface))

SURFACE: 101, PZ, periodic surface: None, transform: None, constants: [-0.63]
SURFACE: 102, PZ, periodic surface: None, transform: None, constants: [0.63]


# Task 1.2: Set Reflective Boundaries
* This is a property of surfaces
* Let's check those docs

## Task set one surface to be reflective
* Reminder: `surface` will be still set from the previous for loop, and should be a `PZ` surface

In [14]:
IFrame("https://www.montepy.org/en/stable/api/montepy.surfaces.surface.html", 800, 500)

# Pause for students

In [15]:
surface.is_reflecting = True
print(surface.mcnp_str())

*102 PZ  0.63


/home/mgale/miniforge3/envs/montepy_demo/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Goal 1: Conclusion: Bring it all together
* Iterate over all `PX`, `PY`, and `PZ` surfaces
* Set all such surfaces to `is_reflecting`

* For the adventerous: check out [`itertools.chain`](https://docs.python.org/3/library/itertools.html#itertools.chain)

# Pause

In [16]:
# Task: Set all PX, PY, and PZ surfaces to reflecting
import itertools
surfs = problem.surfaces
iter_surfaces = itertools.chain(surfs.px, surfs.py, surfs.pz)
for surface in iter_surfaces:
    surface.is_reflecting = True
    print(surface.mcnp_str())

*103 PX -0.63
*104 PX  0.63
*105 PY -0.63
*106 PY  0.63
*101 PZ -0.63
*102 PZ  0.63


/home/mgale/miniforge3/envs/montepy_demo/lib/python3.12/site-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Goal 2: Fill the Gas Gap with Helium

## Tasks
1. Find the gas gap cell
2. create a new helium cell
3. assign that material to the gas gap cell
4. Set the density of the gas gap


# Task 2.1: Finding the Cell

Ways to find a cell:

1. Knowing its cell number A-priori
    * Boring, and error-prone
2. By its comments if they are helpful
3. By its material
    * This is the only void cell
4. By its surfaces
    * This should be sandwiched between the smallest, and next smallest cylinder

In [17]:
# We'll get a cell to play with
# this grabs the first cell
cell = list(problem.cells)[0]

# Explore a cell
## Task, print all of the following for the cell
* its number
* its comment(s)
* its surfaces
* its material

In [18]:
IFrame("https://www.montepy.org/en/stable/api/montepy.cell.html", 800, 600)

# Pause

In [19]:
print("number", cell.number)
print("comments", cell.comments)
print("material", cell.material)
print("surfaces", cell.surfaces)

number 1
comments [COMMENT: C dimensions, densities, and material compositions are based on the BEAVRS benchmark, COMMENT: C They have been rounded, and further simplified, COMMENT: C
, COMMENT: C N. Horelik, B. Herman, B. Forget, and K. Smith. Benchmark for Evaluation and Validation of, COMMENT: C    Reactor Simulations (BEAVRS), v1.0.1. Proc. Int. Conf. Mathematics and Computational Methods, COMMENT: C    Applied to Nuc. Sci. & Eng., 2013. Sun Valley, Idaho, COMMENT: c
, COMMENT: c uranium rod]
material MATERIAL: 1, ['oxygen', 'uranium']
surfaces Surfaces: [1, 101, 102]


# Task 2.1: Find the cell which is Void (material is `None`)

# Pause

In [20]:
# Task: set 'gap' to the void Cell
gap = None
for cell in problem.cells:
    if cell.material is None:
        gap = cell
        break
if gap is not None:
    print(gap, gap.comments)

CELL: 2, mat: 0, DENS: None [COMMENT: c gas gap]


# Task 2.2 Define a Helium Material

## Steps
1. Create a new material
2. Request a number for the material
3. Add it to the problem

In [21]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html",
    800,
    600,
)

In [22]:
IFrame("https://www.montepy.org/en/stable/api/montepy.materials.html", 800, 600)

# Pause

In [23]:
# Task: define 'helium' Material and add it to the problem
helium = montepy.Material()
if helium is not None and helium not in problem.materials:
    helium.number = problem.materials.request_number()
    problem.materials.append(helium)

# Adding Nuclide
* Will model as pure `He-4`
* Need to find appropriate ACE file
* Want to use ENDF/B-VII.1
* 600 K is about right
* Refer to the ACE manual [LA-UR-17-20709](https://doi.org/10.2172/1342828)
   * Hint: this is `81c`
 
## Note on Atomic/mass fraction
---
* Whether a material is in atomic or mass fraction is  a property of a material

# Pause

In [24]:
?montepy.Material.add_nuclide

Signature:
montepy.Material.add_nuclide(
    self,
    nuclide: 'NuclideLike',
    fraction: 'float',
)
Docstring:
Add a new component to this material of the given nuclide, and fraction.

.. versionadded:: 1.0.0

Parameters
----------
nuclide : Nuclide, str, int
    The nuclide to add, which can be a string Identifier, or
    ZAID.
fraction : float
    the fraction of this component being added.
File:      ~/miniforge3/envs/montepy_demo/lib/python3.12/site-packages/montepy/data_inputs/material.py
Type:      function

In [25]:
# Task: Add one or more helium isotopes to the 'helium' 
# Hint: the ENDF/B-VII.1 600K continuous-energy library is "81c"
nuc = montepy.Nuclide("He-4.81c")
if nuc is not None:
    helium.add_nuclide(nuc, 1.0)
    print(helium.is_atom_fraction)

True


# Task 2.3: Finding Density
* PWR helium gas gap pressure starts at around 25 atm (according to [nuclear-power.com](https://www.nuclear-power.com/nuclear-power-plant/nuclear-fuel/fuel-assembly/fuel-rods-fuel-pins/))
* From [Ideal gas law](https://en.wikipedia.org/wiki/Ideal_gas_law)
 $$ p = \frac{nRT}{N_A}$$
* With:
    * $p$ being the pressure. 
    * $n$ being the atomic density
    * $R$ is the ideal gas constant. You can use:$8.2057\times10^{-5}\rm\frac{m^3atm}{K\cdot mol}$
    * $N_A$ is Avogadro's constant. You can use: $ 6.022\times 10^{23}$
    * $T$ is the temperature. Let's assume 600 K.
* Solve for the atom density, and convert it to units of $\rm\frac{a}{barn\cdot cm}$ (1 barn = $1\times10^{-24}\rm cm^2$)

# pause

 $$ p = \frac{nRT}{N_A}$$

In [26]:
R = 8.2057e-5  # m3*atm/K-mol
N_A = 6.022e23
BARNS_TO_CM2 = 1e24
PRESSURE = 25  # atm
TEMP = 600  # K

In [27]:
density_m3 = (PRESSURE * N_A) / (R * TEMP)
density_cm3 = density_m3 / (100**3)  # (cm -> m)^3
density = density_cm3 / BARNS_TO_CM2
density = np.round(density, 5)
print(f"{density:.3e} at/b-cm")

3.100e-04 at/b-cm


# Task 2.4: Bring it all together
* Assign cell material
* Assign density

In [28]:
IFrame("https://www.montepy.org/en/stable/api/montepy.cell.html", 800, 600)

# Pause

In [29]:
gap.material = helium
gap.atom_density = density
gap

CELL: 2 
MATERIAL: 3, ['helium']
density: 0.00031 atom/b-cm
SURFACE: 1, CZ
SURFACE: 2, CZ
SURFACE: 101, PZ
SURFACE: 102, PZ

# Goal 3: Fix the Zircalloy Material definition
* Find the zircalloy
* Remove elemental nuclides
* Replace with proper isotopic break downs

## Task 3.1 find Zircalloy
* Easy to find as only material with Zr
* `Materials` have some helpful functions for this

In [30]:
IFrame("https://www.montepy.org/en/stable/api/montepy.materials.html#montepy.materials.Materials.get_containing_any", 800, 600)

# Pause

In [31]:
# Task: Set 'zirc' to the material containing Zirconium (Atomic symbol: Zr; Z: 40)
zirc = next(problem.materials.get_containing_any("Zr"))
if zirc is not None:
    print(repr(zirc))

MATERIAL: 10 fractions: mass
Zr-0     (66c) 0.98
Sn-0     (42c) 0.015
Fe-0     (55c) 0.002
Cr-0     (50c) 0.001



# Task 3.2 Define new components

* take existing components
* Split into isotopic abundances and multiply with the existing components
* Remove old components

## Note
---
* MontePy does not provide natural isotopic abundances
    * Currently don't have resources to maintain these data
    * Would need to be able to read your `XSDIR` file
* OpenMC can provide you with these data

In [32]:
NAT_ABUNDANCES = {
    "Zr": {90: 0.5145, 91: 0.1122, 92: 0.1715, 94: 0.1738, 96: 0.0280},
    "Sn": {112: 0.0097, 114: 0.0066, 116: 0.0034, 117: 0.0768},
    "Fe": {54: 0.05845, 56: 0.91754, 57: 0.02119, 58: 0.00282},
    "Cr": {50: 0.04345, 52: 0.83789, 53: 0.09501, 54: 0.02365},
}
# From Meija, et al. "Isotopic Compositions of the Elements (IUPAC Technical Report) <https://doi.org/10.1515/pac-2015-0503>
# given in atom fraction.

# Play around with Materials
* Review [Material documentation](https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html)
* Adding to a list while you are iterating over it is dangerous; make sure to:
    1. Create a new list of the components
    2. Clear the old material
    3. Add the new components to the material.
    4. Change libraries for all nuclides all at once.

# Pause

In [33]:
# Task: Replace all components of material 'zirc'
zirc2 = zirc.clone()
zirc2.clear()
for nuclide, base_fraction in zirc:
    element = nuclide.element
    abundances = NAT_ABUNDANCES[element.symbol]
    for A, iso_fraction in abundances.items():
        print(f"A: {A:3g} | fraction: {iso_fraction:}")
        # Now, add it to our new zirc2:
        print(f"{element.symbol}-{A}")
        zirc2.add_nuclide(f"{element.symbol}-{A}",base_fraction * iso_fraction)
zirc2.change_libraries("82c")
zirc2

A:  90 | fraction: 0.5145
Zr-90
A:  91 | fraction: 0.1122
Zr-91
A:  92 | fraction: 0.1715
Zr-92
A:  94 | fraction: 0.1738
Zr-94
A:  96 | fraction: 0.028
Zr-96
A: 112 | fraction: 0.0097
Sn-112
A: 114 | fraction: 0.0066
Sn-114
A: 116 | fraction: 0.0034
Sn-116
A: 117 | fraction: 0.0768
Sn-117
A:  54 | fraction: 0.05845
Fe-54
A:  56 | fraction: 0.91754
Fe-56
A:  57 | fraction: 0.02119
Fe-57
A:  58 | fraction: 0.00282
Fe-58
A:  50 | fraction: 0.04345
Cr-50
A:  52 | fraction: 0.83789
Cr-52
A:  53 | fraction: 0.09501
Cr-53
A:  54 | fraction: 0.02365
Cr-54


MATERIAL: 4 fractions: mass
Zr-90    (82c) 0.5042099999999999
Zr-91    (82c) 0.109956
Zr-92    (82c) 0.16807
Zr-94    (82c) 0.170324
Zr-96    (82c) 0.02744
Sn-112   (82c) 0.00014549999999999999
Sn-114   (82c) 9.9e-05
Sn-116   (82c) 5.099999999999999e-05
Sn-117   (82c) 0.0011519999999999998
Fe-54    (82c) 0.00011690000000000001
Fe-56    (82c) 0.00183508
Fe-57    (82c) 4.2380000000000004e-05
Fe-58    (82c) 5.64e-06
Cr-50    (82c) 4.3450000000000006e-05
Cr-52    (82c) 0.00083789
Cr-53    (82c) 9.501e-05
Cr-54    (82c) 2.3650000000000002e-05

In [ ]:
# TODO set cell to zirc2
#TODO delete zirc

# Task 4 Set Up Eigenvalue run

* Need to define [ksrc](https://mcnp.lanl.gov/pdf_files/TechReport_2022_LANL_LA-UR-22-30006Rev.1_KuleszaAdamsEtAl.pdf#subsection.5.8.11), and [kcode](https://mcnp.lanl.gov/pdf_files/TechReport_2022_LANL_LA-UR-22-30006Rev.1_KuleszaAdamsEtAl.pdf#subsection.5.8.10).
* MontePy doesn't support `KCODE` yet directly, but can be given an arbitrary input string and add it to the model.
  *  _**Note: I strongly recommend putting skeleton code with comments telling the students exactly what to do.**_

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html#montepy.mcnp_problem.MCNP_Problem.parse",
    800,
    600,
)

# Task 4.1
* Set one source site at the origin
* Set:
    * 100,000 histories per cycle
    * initial guess of k to be 1.1
    * Do 100 cycles
    * with 20 inactive cycles.

# Pause

In [ ]:
# Task: Add kcode and ksrc to the problem
nparticles = int(1e5)
kguess = 1.1
inactive_batches = 20
total_batches = 100
kcode = problem.parse(f"kcode {nparticles} {kguess} {inactive_batches} {total_batches}", append=False)
# Now add a ksrc
ksrc = problem.parse("ksrc 0 0 0 $ source in center of fuel pin", append=False)

In [ ]:
# If 'append=False' earlier, append to the data block now.
problem.data_inputs.append(kcode)
problem.data_inputs.append(ksrc)

# Task 5 update water Density
* Moderator density is around standard temperature and pressure (roughly)
* Much too dense for a PWR
* Density should be 0.74 $\rm\frac{g}{cm^3}$ <sup>1</sup>

<sup>1</sup> N. E. Horelik et al., "Benchmark for Evaluation and Validation of Reactor Simulations (BEAVRS)," presented at the Int. Conf. Mathematics and Computational Methods Applied to Nuc. Sci. & Eng., Sun Valley, Idaho, 2013.

In [ ]:
WATER_DENSITY = 0.74  # [g/cm3]

# Task 5.1 Find moderator cell
* will find water material, then find the cell filled with that

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html#montepy.materials.Materials.get_containing_all",
    800,
    600,
)

# Pause

In [ ]:
# get water
water = list(problem.materials.get_containing_all("H", "O"))[0]
water

# Grab Cell

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html#montepy.data_inputs.material.Material.cells",
    800,
    600,
)

# Pause

In [ ]:
water_cell = list(water.cells)[0]
water_cell

# Set Cell Mass Density

In [ ]:
IFrame(
    "https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.mass_density",
    800,
    600,
)

# Pause

In [ ]:
water_cell.mass_density = WATER_DENSITY

# Conclusion
* Write it out to a file `models/pin_cell_corrected.imcnp`

In [ ]:
IFrame("https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html", 800, 600)

# Pause

In [ ]:
problem.write_problem("models/pin_cell_corrected.imcnp")

In [ ]:
!cat models/pin_cell_corrected.imcnp

# Questions?